<a href="https://colab.research.google.com/github/mahima-c/Fake-New-Prediction-LSTM/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
! ls

deep-learning  Fake-New-Prediction-LSTM  nn  train.csv


In [ ]:
%cd gdrive/My Drive/project_folder/Fake-New-Prediction-LSTM/


[Errno 2] No such file or directory: 'gdrive/My Drive/project_folder/Fake-New-Prediction-LSTM/'
/content/gdrive/My Drive/project_folder


In [ ]:
pwd

'/content/gdrive/My Drive/project_folder'

In [ ]:
import pandas as pd
df=pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.shape

(20800, 5)

In [ ]:
###Drop Nan Values
df=df.dropna()

In [ ]:
df.shape

(18285, 5)

In [ ]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [ ]:
## Get the Dependent features
y=df['label']

In [ ]:
import tensorflow as tf


In [ ]:
tf.__version__


'2.4.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense,Dropout

In [ ]:
### Vocabulary size
voc_size=5000

Onehot **Representation**


In [ ]:
messages=X.copy()

In [ ]:
messages['title'].head(10)
messages.index
if 6 not in messages.index:
  print("4")


4


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# # dataset preprocessing
# from nltk.stem.porter import PorterStemmer
# ps = PorterStemmer()
# corpus = []
# for i in range(0, len(messages)):
#   if i in messages.index:
#     print(i)
#     review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
#     review = review.lower()
#     review = review.split()
    
#     review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
#     review = ' '.join(review)
#     corpus.append(review)

In [ ]:
# dataset preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for sents in messages['title']:
  # if i in messages.index:
    # print(i)
    review = re.sub('[^a-zA-Z]', ' ', sents)
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
len(corpus)

18285

In [ ]:
# corpus

In [ ]:
one_hot(corpus[1],1500)

[117, 528, 370, 531, 711, 344, 1471]

In [ ]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [ ]:
onehot_repr=[one_hot(sents,voc_size)for sents in corpus] 
onehot_repr[:10]

[[216, 4593, 3696, 786, 3949, 1764, 4235, 4573, 3808, 602],
 [194, 415, 131, 2935, 1978, 4008, 1908],
 [3398, 4603, 3376, 1763],
 [3730, 1201, 2737, 3776, 3477, 4702],
 [205, 1978, 277, 10, 1976, 48, 1978, 4734, 240, 3799],
 [3975,
  3355,
  1181,
  4516,
  680,
  3828,
  3507,
  1010,
  1090,
  2300,
  3389,
  1773,
  1355,
  3329,
  1908],
 [420, 3074, 2223, 2719, 140, 927, 1956, 4190, 2221, 1475, 2590],
 [1572, 2580, 506, 48, 4164, 3544, 3828, 4529, 2221, 1475, 2590],
 [2307, 2607, 2157, 2813, 1099, 3687, 2752, 3558, 3828, 1034],
 [4553, 2866, 672, 2603, 861, 4857, 3928, 2195]]

Embedding Representation


In [ ]:
# first find max length
def findMaxLength(onehot_repr):
  max=0
  for item in onehot_repr:
    # print(len(item))
    if len(item) > max:
      max=len(item)
  return max

In [175]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=47)
print(embedded_docs[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0  216 4593 3696  786 3949
 1764 4235 4573 3808  602]


In [176]:
## Creating mode
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100,activation='relu'))
# model.add(Dropout(0.3))

model.add(Dense(1,activation='sigmoid'))
# model.add(Dropout(0.3))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 20, 40)            200000    
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               56400     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [177]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 7s 27ms/step - loss: 0.4906 - accuracy: 0.7490 - val_loss: 0.2003 - val_accuracy: 0.9145
Epoch 2/10
192/192 [==============================] - 5s 26ms/step - loss: 0.1611 - accuracy: 0.9407 - val_loss: 0.1960 - val_accuracy: 0.9183
Epoch 3/10
192/192 [==============================] - 5s 26ms/step - loss: 0.1108 - accuracy: 0.9620 - val_loss: 0.2156 - val_accuracy: 0.9153
Epoch 4/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0680 - accuracy: 0.9759 - val_loss: 0.2445 - val_accuracy: 0.9195
Epoch 5/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0484 - accuracy: 0.9823 - val_loss: 0.3022 - val_accuracy: 0.9175
Epoch 6/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0268 - accuracy: 0.9922 - val_loss: 0.3368 - val_accuracy: 0.9163
Epoch 7/10
192/192 [==============================] - 5s 29ms/step - loss: 0.0157 - accuracy: 0.9957 - val_loss: 0.5185 - val_accuracy: 0.9135

In [180]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=47))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [184]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0044 - accuracy: 0.9983 - val_loss: 0.5487 - val_accuracy: 0.9143
Epoch 2/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0042 - accuracy: 0.9989 - val_loss: 0.5718 - val_accuracy: 0.9074
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0044 - accuracy: 0.9985 - val_loss: 0.6537 - val_accuracy: 0.9100
Epoch 4/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0046 - accuracy: 0.9981 - val_loss: 0.6036 - val_accuracy: 0.9075
Epoch 5/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0042 - accuracy: 0.9989 - val_loss: 0.6031 - val_accuracy: 0.9065
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0035 - accuracy: 0.9993 - val_loss: 0.6254 - val_accuracy: 0.9135
Epoch 7/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0038 - accuracy: 0.9990 - val_loss: 0.5215 - val_accuracy: 0.9118

In [185]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)
X_final.shape,y_final.shape

((18285, 47), (18285,))

In [186]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [187]:
# Performance Metrics And Accuracy
y_pred=model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[3053,  366],
       [ 140, 2476]])

In [188]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9161557580778791

In [192]:
from sklearn.externals import joblib
joblib.dump(model, 'model_joblib')



/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


TypeError: ignored

# **Model** **save**

In [193]:
import os.path
if os.path.isfile('models/fake_news.h5') is False:
  model.save('models/fake_news.h5')

This save functions saves:


1.   architecture of the model,allowing to re-create the model
2.   weight of model
3.   the training configuration(loss,optimizer)
4.   the state of optimizer,allowing to resume traing excatly where you left of



In [194]:
!ls

deep-learning		  fake_news_model_pickle  models  train.csv
Fake-New-Prediction-LSTM  model_joblib		  nn


In [195]:
from tensorflow.keras.models import load_model
new_model=load_model('models/fake_news.h5')

In [196]:
new_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 47, 40)            200000    
_________________________________________________________________
dropout_14 (Dropout)         (None, 47, 40)            0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               56400     
_________________________________________________________________
dropout_15 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [197]:
new_model.get_weights()

[array([[-2.3835180e-02, -3.1834044e-02,  2.8526124e-02, ...,
         -4.0066496e-02, -1.2714627e-02, -5.7906308e-03],
        [-8.5269064e-02, -2.4857359e-02,  7.0802458e-02, ...,
         -8.7597452e-02, -4.5441389e-02, -3.9584156e-02],
        [ 9.6829683e-02,  2.5984667e-02,  1.0150540e-02, ...,
          1.0941322e-01,  2.4698395e-02, -2.0096730e-03],
        ...,
        [ 6.8492927e-02,  1.7268892e-02, -1.3178441e-02, ...,
          9.9657252e-03,  4.8860870e-02,  2.7084744e-02],
        [-2.7749099e-02, -3.4765087e-02,  2.7330836e-02, ...,
         -6.8411998e-02, -1.6873887e-02, -4.5585454e-02],
        [ 6.4097205e-03,  5.0837848e-02,  1.0712799e-04, ...,
         -1.3412867e-01,  1.9378684e-02,  7.7384979e-02]], dtype=float32),
 array([[ 0.32583502, -0.05737324, -0.16181287, ...,  0.25696632,
          0.12673953,  0.8141447 ],
        [ 0.08897047,  0.25816366, -0.06705058, ...,  0.12162565,
          0.21378072, -0.01250493],
        [ 0.14081204, -0.10256518, -0.1900523 

In [199]:
new_model.optimizer

## **2.model.to_json()**

if only save architure of model not weight of model or training configuration,then you can use this function

In [200]:
# save as json
json_string=model.to_json()

# save as YAML
# yaml_string=model.to_yaml()

In [201]:
json_string

'{"class_name": "Sequential", "config": {"name": "sequential_11", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 47], "dtype": "float32", "sparse": false, "ragged": false, "name": "embedding_11_input"}}, {"class_name": "Embedding", "config": {"name": "embedding_11", "trainable": true, "batch_input_shape": [null, 47], "dtype": "float32", "input_dim": 5000, "output_dim": 40, "embeddings_initializer": {"class_name": "RandomUniform", "config": {"minval": -0.05, "maxval": 0.05, "seed": null}}, "embeddings_regularizer": null, "activity_regularizer": null, "embeddings_constraint": null, "mask_zero": false, "input_length": 47}}, {"class_name": "Dropout", "config": {"name": "dropout_14", "trainable": true, "dtype": "float32", "rate": 0.3, "noise_shape": null, "seed": null}}, {"class_name": "LSTM", "config": {"name": "lstm_11", "trainable": true, "dtype": "float32", "return_sequences": false, "return_state": false, "go_backwards": false, "stateful": false, "unroll

In [202]:
# model reconstruction from json
from tensorflow.keras.models import model_from_json
model_architecture=model_from_json(json_string)

In [203]:
model_architecture.summary()#only architecture store so we need to train and compile it

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 47, 40)            200000    
_________________________________________________________________
dropout_14 (Dropout)         (None, 47, 40)            0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               56400     
_________________________________________________________________
dropout_15 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


### **3.model.save_ weight()**

if only need to save the weight of model,so we can use the following function save to weight only.

In [204]:
# check first file exist already or not
import os.path
if os.path.isfile('models/fake_news.h5') is False:
  model.save('models/fake_news_weights.h5')

In [206]:
# this weigth is used where you need them,if we create same architecture of another model then 
# after create model load with these weigth(using load_weights)